In [143]:
from scipy.stats import norm
import numpy as np
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress = True)

In [144]:
#Init parameters
S0_init = 30       #S_0
K_init = 30        #Strike price (=S_0)
T_init = 10        #Maturity
v_init = 3         #Vesting period
r_init = 0.05      #RF rate
N_init = 10        #Height of tree
sigma_init = 0.3   #Volatility
m_init = 2         #Exercise multiple (m>1)

#For R option: alpha is percentage that gets converted to stock, gamma the "risk premium"
alpha_init = 0.9
gamma_init = 0.05

In [145]:
def rn_eso(S0,K,T,v,r,N,sigma,m):
  #Init values
    dt = 1/N                        #number of steps
    u = np.exp(sigma * np.sqrt(dt)) #using CRR method with (constant) volatility
    d = 1/u                         #to maintain the triangular structure of the tree (i.e., recombinant tree)
    q = (np.exp(r*dt) - d)/(u-d)    #q is the RN probability
    disc = np.exp(-r*dt)            #discount

  #Build up terminal stock price nodes
    S = np.zeros(T*N+1)
    for j in range(0, T*N+1): #build up the nodes from the bottom
      S[j] = S0 * u**j * d**(T*N-j)

  #Option payoff if exercising at all nodes
    C = np.zeros(T*N+1)
    for j in range(0, T*N+1):
      C[j] = max(0, S[j] - K)

  #Backward recursion through the tree: at each node, is it optimal to exercise or not?
    for i in np.arange(T*N-1,-1,-1):
      for j in range(0,i+1):
        S = S0 * u**j * d**(i-j)                      #S is function of j (#ups) and i-j (#downs)
        vested = (i+j >= v*N)

        if not vested:                                #Unvested
          C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
        elif vested & (S>=K*m):                       #Vested and early exercisable (as function of multiple - )
          C[j] = S - K
        elif vested & (S<K*m):                        #Vested but unexercisable (as function of multiple)
          C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )

        #NON-VESTED case - Comment the ifs above and run the below
          #C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
          #C[j] = max(C[j], S - K)
    return C[0]

In [161]:
def black_scholes(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma**2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [164]:
def r_eso(S0,K,T,v,r,N,sigma,m,alpha,gamma):
  #Init values
    dt = 1/N                        #number of steps
    u = np.exp(sigma * np.sqrt(dt)) #using CRR method with (constant) volatility
    d = 1/u                         #to maintain the triangular structure of the tree (i.e., recombinant tree)
    q = (np.exp(r*dt) - d)/(u-d)    #q is the RN probability
    disc = np.exp(-r*dt)            #discount


  #Build up stock price tree (needed for next step)
    S = np.zeros(T*N+1)
    for j in range(0, T*N+1): #build up the nodes from the bottom
      S[j] = S0 * u**j * d**(T*N-j)

  #Initialize option vector
    C = np.zeros(T*N+1)
    for j in range(0, T*N+1):
      C[j] = max(0, S[j] - K)

  #Backward recursion through the tree: at each node, is it optimal to exercise or not?
    for i in np.arange(T*N-1,-1,-1):
      for j in range(0,i+1):
        S = S0 * u**j * d**(i-j)                      #S is function of j (#ups) and i-j (#downs)
        vested = (i+j >= v*N)

        if not vested:                                  #Unvested
          C[j] = disc * (q*C[j+1] + (1-q)*C[j])
        elif vested & (S >= K*m):                       #Vested and exercisable (as function of multiple)
          C[j] = S - K + (1-alpha+gamma)*(black_scholes(S,K,T,r,sigma)-(S-K))
        elif vested & (S < K*m):                        #Vested but unexercisable (as function of multiple)
          C[j] = disc * (q*C[j+1] + (1-q)*C[j])

        #NON-VESTED case - Comment the ifs above and run the below
          #C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
          #C[j] = max(C[j], S - K)

    return C[0]

In [148]:
def r_eso_mod(S0,K,T,v,r,N,sigma,m,alpha,gamma):
  #Init values
    dt = 1/N                        #number of steps
    u = np.exp(sigma * np.sqrt(dt)) #using CRR method with (constant) volatility
    d = 1/u                         #to maintain the triangular structure of the tree (i.e., recombinant tree)
    q = (np.exp(r*dt) - d)/(u-d)    #q is the RN probability
    disc = np.exp(-r*dt)            #discount


  #Build up stock price tree (needed for next step)
    S = np.zeros(T*N+1)
    for j in range(0, T*N+1): #build up the nodes from the bottom
      S[j] = S0 * u**j * d**(T*N-j)

  #Option payoff if exercising at all nodes
    C = np.zeros(T*N+1)
    for j in range(0, T*N+1):
      C[j] = max(0, S[j] - K)

  #Backward recursion through the tree: at each node, is it optimal to exercise or not?
    for i in np.arange(T*N-1,-1,-1):
      for j in range(0,i+1):
        S = S0 * u**j * d**(i-j)                      #S is function of j (#ups) and i-j (#downs)
        vested = (i+j >= v*N)

        if not vested:                                #Unvested
          C[j] = disc * (q*C[j+1] + (1-q)*C[j])
        elif vested & (S >= K*m):                       #Vested and exercisable (as function of multiple)
          C[j] = (1+gamma)*(S - K) + (1-alpha+gamma)*(black_scholes(S,K,T,r,sigma)-(S-K))
        elif vested & (S < K*m):                        #Vested but unexercisable (as function of multiple)
          C[j] = disc * (q*C[j+1] + (1-q)*C[j])

        #NON-VESTED:
          #C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
          #C[j] = max(C[j], S - K)

    return C[0]

In [149]:
rn = rn_eso(S0_init,K_init,T_init,v_init,r_init,1,sigma_init,m_init)
print("Cost of RN:", rn)

Cost of RN: 13.655803695861094


In [182]:
r = r_eso(S0_init,K_init,T_init,v_init,r_init,500,sigma_init,m_init,alpha_init,gamma_init)
print("Cost of R: ", end="")
print(r)

Cost of R: 13.408953433662534


In [151]:
r_mod = r_eso_mod(S0_init,K_init,T_init,v_init,r_init,N_init,sigma_init,m_init,alpha_init,gamma_init)
print("Cost of R modified: ", end="")
print(r_mod)

Cost of R modified: 14.270431106226274


In [152]:
gammas = [0, 0.1, 0.2, 0.3]
for g in gammas:
  #Compute costs:
  rn = rn_eso(S0_init,K_init,T_init,v_init,r_init,N_init,sigma_init,m_init)
  r = r_eso(S0_init,K_init,T_init,v_init,r_init,N_init,sigma_init,m_init,alpha_init,g)
  r_mod = r_eso_mod(S0_init,K_init,T_init,v_init,r_init,N_init,sigma_init,m_init,alpha_init,g)

  print(g)
  print("Cost of RN: ", end="")
  print(rn)

  print("Cost of R: ", end="")
  print(r)

  print("Cost of R mod: ", end="")
  print(r_mod)


  print("R - RN = ", end="")
  print(r-rn)
  print("R_mod - R = ", end="")
  print(r_mod-r)
  print("\n")

0
Cost of RN: 12.981051280090961
Cost of R: 13.436197831881772
Cost of R mod: 13.436197831881772
R - RN = 0.4551465517908113
R_mod - R = 0.0


0.1
Cost of RN: 12.981051280090961
Cost of R: 13.891344383672584
Cost of R mod: 15.10466438057078
R - RN = 0.9102931035816226
R_mod - R = 1.2133199968981963


0.2
Cost of RN: 12.981051280090961
Cost of R: 14.34649093546339
Cost of R mod: 16.773130929259793
R - RN = 1.3654396553724286
R_mod - R = 2.426639993796403


0.3
Cost of RN: 12.981051280090961
Cost of R: 14.801637487254203
Cost of R mod: 18.441597477948793
R - RN = 1.8205862071632417
R_mod - R = 3.6399599906945905




Next steps:
*   [DONE] Plot # simulations needed for each option (1, 5, 10, 100)

*   Plot R vs R_mod
    * function of different alphas and gammas (steps by 0.01)
*   Plot R, R_mod as function of changing S0, K, T,
    * Expected: value goes up with higher S0, T, sigma, alpha, gamma
    * Expected: value goes down with higher K, v,
    * Effect of m (?)


*   Compute two Greeks 
    * Delta: >0 because call option - changes in underlying price
        * Graph Delta - alpha; can overlap graphs (2 changing vars needed)
        * Graph Delta - gamma; can overlap graphs (2 changing vars needed)
    * (Vega - changes in volatility; same graphs as above)


*   !! Re-run everything using trinomial model
*   To check: RN should be equal to R with alpha=1, gamma=0!!


FROM HERE BELOW, ONLY PROD CODE 


In [174]:
#PROD: REPLICATION OF TESO

rn = rn_eso(30,30,10,3,0.05,1,0.3,2)
print("Cost of TESO:", rn)

Cost of TESO: 13.655803695861094


In [175]:
#PROD: REPLICATION OF DESO

r = r_eso(30,30,10,3,0.05,1,0.3,2,0.75,0)
print("Cost of R:", r)

Cost of R: 14.553842384081651


In [180]:
#PROD: TABLE diff_n - 2h to run

N = [1, 10, 50, 100, 365, 500, 1000]

rns = [rn_eso(30,30,10,3,0.05,n,0.3,2) for n in N]
rs = [r_eso(30,30,10,3,0.05,n,0.3,2,0.75,0) for n in N]

rns =  ['%.2f' % elem for elem in rns]
rs = ['%.2f' % elem for elem in rs]


print(rns)
print(rs)

['13.66', '12.98', '12.79', '12.79', '12.66', '12.69']
['14.55', '14.12', '14.00', '14.00', '13.91', '13.93']
